In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
path="../input/just-test/hello - pluviometrie.csv"
data=pd.read_csv(path)
data.head()

In [3]:
data.shape # shape before Cleaning

In [4]:
data = data.drop_duplicates() # pour supprimer les lignes dupliquées

In [5]:
data.shape #shape after cleaning

In [6]:
data.info()

In [7]:
data.isnull().sum() #pour calculer le nombre de NaN dans chaque colone

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [9]:
data['station'].nunique() #calculer le nombre de stations


On a 127 stations comme le montre la derniére ligne du dernier output.


///////////////////////////////////////////////////////////////////////////////////////


In [10]:
from sklearn.preprocessing import LabelEncoder
staation = LabelEncoder()
data['station'] = staation.fit_transform(data['station'])
data["station"].unique()

**on a 127 stations qu'on numérote de 0 vers 126**


In [11]:
fig = plt.figure(figsize = (15,15))
plt.scatter(data["Date"][1:3567],data["Pluvio_du_jour"][1:3567],color="red")
plt.title('Pluviométrie en Tunisie durant le mois de Juillet 2021')
plt.xlabel("date")
plt.xticks(rotation=90)
plt.ylabel("pluviométrie du jour")
plt.show()

Généralement pas de pluie en mois de Juillet à l'exception de quelques jours et ceci avec des quantités faibles.
NB: chaque point dans le graphe représente une station, un seul point en 0 veut dire qu'il n'y avait pas de pluie dans toutes les stations.

In [12]:
dataTypeSeries = data.dtypes
print('Data type of each column of Dataframe :')
print(dataTypeSeries)

il faut transformer la variable Date à une valeur numérique. Pour cela on définit la fonction ci-dessous

In [13]:
def transform(d):
    l=d.split("-",maxsplit=2)
    int_list = list(map(int, l))

    return  int_list[0]*10000+int_list[1]*100+int_list[2]

In [14]:
data['Date'] = data['Date'].apply(transform)

L'input ci-dessus applique la fonction transform sur la colone date pour transformer les dates en des entiers et l'input ci-dessous vérifie que se s'est passée correctement


In [15]:
data.head()

In [16]:
data.tail()

In [17]:
X = data.drop(["Cumul_du_mois","Pluvio_du_jour"] ,axis=1) #features
y = data["Cumul_du_mois"] #label ( on veut prédire la pluviométrie d'un mois)

on choisit de prédire la pluviométrie du mois


In [18]:
###########

In [19]:
X

In [20]:
y

* **** Régression linéaire *****

In [21]:
from sklearn.linear_model import LinearRegression # regression linéaire
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y.values, random_state = 0)
#la ligne ci-dessus divise les données en données pour entrainement et pour test
linear_reg = LinearRegression()
linear_reg.fit(train_X, train_y)

In [22]:
y_pred = linear_reg.predict(val_X)

In [23]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
error = np.sqrt(mean_squared_error(val_y, y_pred))

In [24]:
error

On trouve une erreur de 23.9 mm par mois pour la regression linéaire. On essaie alors d'autres algorithmes.


**decision tree regressor ( ARBRE de décision)****


In [25]:
from sklearn.tree import DecisionTreeRegressor
dec_tree_reg = DecisionTreeRegressor(random_state=0)
dec_tree_reg.fit(train_X, train_y)

In [26]:
y_pred = dec_tree_reg.predict(val_X)

In [27]:
error = np.sqrt(mean_squared_error(val_y, y_pred))
error 

On trouve une erreur de seulement 3.94 pour la pluviométrie d'un mois ( l'algorithme de l'arbre de décision est beaucoup plus meilleur)


In [28]:
from sklearn import tree
#tree.plot_tree(dec_tree_reg) 

**Random forrest regression ( foret aléatoire)****

In [29]:
from sklearn.ensemble import RandomForestRegressor 
# on essaye maintenant l'algorithme de RandomForrest
random_forest_reg = RandomForestRegressor(random_state=0)
random_forest_reg.fit(train_X, train_y)

In [30]:
y_pred = random_forest_reg.predict(val_X)

In [31]:
error = np.sqrt(mean_squared_error(val_y, y_pred))
error 

une erreur encore moins importante de seulement 2.78 mm par mois avec l'algorithme foret aléatoire


**Un petit test**


In [32]:
X=np.array([[20210731,65,5.19799998268485,371.2,461.8670773212495,0.0,411.59999999999997 ]])

# 65 désigne la station jendouba (جندوبة)
# mois de juillet

In [33]:
BestRegressor=random_forest_reg # on a obtenu le meilleur résultat avec le random forrest regressor

In [34]:
y_pred = BestRegressor.predict(X)
y_pred

bon résultat : il n'y avait pas de pluie à la station Jendouba en mois de Juillet et cet algorithme de foret aléatoire a juste prédit une pluviométrie de 0.42. +++++++++++
Le vrai résultat est donné ci-dessous

In [35]:
data.head(1)

**********************************************************************************************************
***************************Enregistrement du modéle *********************************

In [36]:
import pickle

In [37]:
dataa = {"model": BestRegressor, "staation": staation}
with open('saved_steps.pkl', 'wb') as file:  # wb : write in binary
    pickle.dump(dataa, file)

In [38]:
with open('saved_steps.pkl', 'rb') as file:
    dataa = pickle.load(file)

regressor_loaded = dataa["model"]
staation = dataa["staation"]


In [39]:
y_pred = regressor_loaded.predict(X)
y_pred 

on trouve le meme résultat donc le modéle a été enregistré avec succés


In [40]:
###############

On teste le modéle sur un autre exemple ( moi de **juillet** station (**101**) **Ghardimaou** ou il y avait juste **3 mm** de puie)


In [41]:
X1=np.array([[20210731,101,3.3177777916193008,338.3,419.5121215622385,0.0,363.9 ]])
y_pred = regressor_loaded.predict(X1)
y_pred 

*** une erreur de 1.9 mm. *** 


In [42]:
###########################################

**Le modéle prédit-il les innondations ?**

En septembre 2020, les stations de Sousse(87), Monastir(32), Mahdia(34) ont eu respectivement comme un cumul de pluie à la fin du mois des valeurs de 194 , 215 et 225.5 mm.

D'aprés http://www.mementodumaire.net/guide-PCS-inondation/PDF/Partie2.pdf un cumul de 50 mm par jour peut se considérer comme inondation dans une zone non montagneuse

In [43]:
# cas station monastir 
X1=np.array([[20200930,32,45.75,411.0,338.12577463482364,31,31 ]])
y_pred = regressor_loaded.predict(X1)
y_pred 

oui, le modéle modéle prédit un cumul de mois de 261 mm ( la valeur réelle est de 215 mm) . le modéle prédit donc l'innondation de la station de Monastir


In [44]:
# cas station mahdia 
X2=np.array([[20200930,34,41.97,316.5,342.982,14.3,14.3 ]])
y_pred = regressor_loaded.predict(X2)
y_pred 

le modéle prédit un cumul de mois de 224 mm ( la valeur réelle est de 225.5 mm) . le modéle prédit donc l'innondation de la station de mahdia


In [45]:
# cas station sousse
X3=np.array([[20200930,87,40.04489793582839,356.3,353.9066720711686,40,40 ]])
y_pred = regressor_loaded.predict(X3)
y_pred 

une prédiction de 140.5 mm par mois ( la valeur réelle est de 194 mm ) on peut considérer que le modéle prédit une faible inondation càd des quantités importantes mais un peu moins importantes que celles des stations Monastir et mahdia

In [46]:
#############

**Prédiction pour le mois de Aout ( voir le tableau Excel)**** **( en utilisant le modéle )**

In [47]:
# cumul de mois prédit par le modéle pour la station Nabeul mois Aout
X=np.array([[20210831,124,10.784,316.7,396.5,0,396.8]])
y_pred = regressor_loaded.predict(X)
y_pred 

In [48]:
####################################################